Analyzing Player Behaviour to Predict Newsletter Supscriptions

By: Gurman Gill

In [3]:
library(tidyverse)
library(tidymodels)
library(readr)
library(janitor)
library(knitr)
library(ggplot2)
library(rmarkdown)
theme_set(theme_minimal())
set.seed(2025)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [4]:
# Introduction

Background: Understanding player behavior is the goal of UBC's Minecraft Research Server Project in order to optimize server resource allocation and engagement tactics.  Whether a player signs up for a newsletter about the game is a crucial engagement indicator.  A player's interest and likelihood of sustained engagement are reflected in their subscriptions, which makes them a useful target for predictive modeling.

Question: Can we predict whether a player will subscribe to the newsletter based on their age, experience level, gender, play duration, and session behavior using KNN classification? 

Data Description: 

We have two datasets provided by the Minecraft Research Server Project team.

players.csv:
- Rows: 196 players
- Columns: 7 variables


sessions.csv:
- Rows - 1535 sessions
- Columns - 5 variables

Below is a table of the relavent variables

| Variable              | Type        | Description                                                    |
|----------------------|-------------|----------------------------------------------------------------|
| `experience`          | Categorical | Self-reported experience level (Amateur, Regular, Pro, etc.)   |
| `subscribe`           | Boolean     | Whether player subscribed to the newsletter                    |
| `hashedEmail`         | Identifier  | Anonymized unique player ID                                    |
| `played_hours`        | Numeric     | Total hours recorded by Minecraft for each player              |
| `name`                | String      | Player's name (not used in modeling)                           |
| `gender`              | Categorical | Player's reported gender                                       |
| `Age`                 | Numeric     | Age of the player (2 values missing originally)                |
| `start_time`          | Numeric     | Start time of sessions                                         |
| `end_time`            | Numeric     | End time of sessions                                           |

Considerations and limitations of data: 
- Self-reported fields like `age`, `gender`, and `experience` may be inaccurate
- Only aggregate features were used; individual session behavior (e.g. time of day, frequency) not explored

In [6]:
# Load Data

players_data <- read_csv("players.csv")
head(players_data)

session_data <- read_csv("

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17
